<img src="data/images/lecture-notebook-header.png" />

# Clustering: DBSCAN

DBSCAN (Density-Based Spatial Clustering of Applications with Noise) is a popular algorithm used for clustering data points based on their density distribution in a given dataset. It is particularly effective in identifying clusters of arbitrary shape and handling outliers or noise points. The DBSCAN algorithm works as follows:

* **Density-Based:** DBSCAN identifies clusters based on the density of data points. It defines density as the number of data points within a specified radius (eps) around each point.

* **Core Points:** DBSCAN starts by randomly selecting a data point and checks if there are at least a minimum number of points (min_samples) within a radius of eps around it. If this condition is satisfied, the point is labeled as a "core point" and becomes the starting point of a new cluster.

* **Directly Density-Reachable:** DBSCAN expands the cluster around a core point by finding other core points that are directly density-reachable from it. A point is considered directly density-reachable if it lies within the eps neighborhood of another core point.

* **Density-Connected:** DBSCAN further expands the cluster by iteratively finding points that are density-reachable from the core points, even if they are not core points themselves. This process continues until no more density-reachable points are found.

* **Border Points:** If a point is not a core point but lies within the eps neighborhood of a core point, it is labeled as a "border point" and may be part of a cluster but does not contribute to the expansion of the cluster.

* **Noise Points:** Data points that are neither core points nor border points are labeled as "noise points" or outliers, as they do not belong to any cluster.

The DBSCAN algorithm does not require specifying the number of clusters in advance, making it more flexible than algorithms like K-means. It can discover clusters of varying shapes and sizes, and it is robust to outliers. However, it does depend on the proper choice of the `eps` and `min_samples` parameters, which can impact the resulting clusters.

**Side note:** This notebook includes different evaluation metrics to assess the quality of clusterings, which will be covered a bit later in the course.

## Setting up the Notebook

### Specify how Plots Get Rendered

In [ ]:
%matplotlib inline

### Import Required Packages

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re

from sklearn.cluster import DBSCAN
from sklearn.datasets import make_blobs, make_moons, make_circles
from sklearn.metrics import silhouette_score, adjusted_rand_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.neighbors import DistanceMetric
from sklearn.metrics.pairwise import euclidean_distances

---

## Playing with Toy Data

[`sklearn.datasets`](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.datasets) provides a series of methods to randomly generate sample data. 

Try different methods and see how the results will change.

In [ ]:
X_demo, y_demo = make_blobs(n_samples=100, centers=5, n_features=2, cluster_std=0.85, random_state=11)
X_demo = X_demo/10 # only needed for make_blobs to have a range from -1 to 1 as well (otherwise the eps value would differ too much for different datasets)

#X_demo, y_demo = make_moons(n_samples=250, noise=0.105, random_state=0)
#X_demo, y_demo = make_circles(n_samples=500, noise=0.06, factor=0.5, random_state=0)

We can plot the data to get a first idea how our data looks like. Of course, in practice this might not be (trivially) possible with data points of more than 3 dimensions. The following example, however, focus on illustrating the characteristics of K-Means.

In [ ]:
plt.figure()

plt.scatter(X_demo[:,0], X_demo[:,1])

plt.show()

The method below plots the clustering, and we will use it throughout the rest of the notebook. The main input of this methods is an instance of [`sklearn.cluster.DBSCAN`](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html) with which the clusters have been calculated.

In [ ]:
def plot_clusters(dbscan, data, point_size=50, show_ticks=True, aspect=None):
    plt.figure()
    
    # Optionally, set aspect ration (only need for lat/lng data)
    if aspect is not None:
        plt.axes().set_aspect(aspect)
    
    # Get the indices of all the points that belong to a cluster (core or boundary point)
    # (the cluster labels go from 0 to (num_clusters-1)
    cluster_point_indices  = np.where(dbscan.labels_ >= 0)[0]
    
    # Get the list of indices of the noise data points (the label of noise is -1)
    noise_indices = np.where(dbscan.labels_ < 0)[0]
    
    # Plot the noise points in black (throws an error if there's no noise)
    # The dots of noise are smaller and plotted first, so cluster points will always be more prominent
    try:
        plt.scatter(data[noise_indices, 0], data[noise_indices, 1], c='black', s=int(point_size/4))    
    except Exception as e:
        pass

    # Plot clusters, each cluster in a different color (at least w.r.t. to the colormap)
    plt.scatter(data[cluster_point_indices, 0], data[cluster_point_indices, 1], c=dbscan.labels_[cluster_point_indices], s=point_size, cmap=plt.cm.tab10)

    # Optionally, remove all ticks and labels
    if show_ticks is False:
        plt.tick_params(top=False, bottom=False, left=False, right=False, labelleft=False, labelbottom=False)  
    
    plt.tight_layout()
    plt.show()
    
dbscan_demo = DBSCAN(eps=0.1, min_samples=8).fit(X_demo) 
plot_clusters(dbscan_demo, X_demo, aspect=1)

Assuming, the blob data, for `eps=0.1` and `min_samples=8`, we can see that some of the outer points of the blobs don't make it into the cluster as the density is to low. These are the **noise points**. Try different parameter values and see how the clustering changes.

### Effects of Different Parameters

With `eps` and `min_samples`, DBSCAN has two core parameters affecting the resulting clustering. So we can use 2 nested loops to vary both parameters over meaningful ranges. For each parameter setting, we perform DBSCAN and keep track of the resulting silhouette score and the (adjusted) rand index.

In [ ]:
silhouette, ari = [], []

for min_samples in np.arange(2, 20, 1):
    for eps in np.arange(0.01, 1.0, 0.01):
        
        # Run DBSCAN for the current parameter values
        dbscan = DBSCAN(eps=eps, min_samples=min_samples).fit(X_demo) 

        # silhouette_score gives the average value for all the samples.
        # This gives a perspective into the density and separation of the formed clusters
        try:
            silhouette.append((min_samples, eps, silhouette_score(X_demo, dbscan.labels_)))
        except Exception as e:
            pass

        ari.append((min_samples, eps, adjusted_rand_score(y_demo, dbscan.labels_)))
    
# Convert to numpy array for convenience
silhouette = np.array(silhouette)    
ari = np.array(ari)

Having two input parameters, we can visualize the results using a 3d plot. Let's first plot the results for the rand index.

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.set_xlabel(r'min_samples', fontsize=16)
ax.set_ylabel(r'eps', fontsize=16)
ax.set_zlabel('Rand Index', fontsize=16)
ax.view_init(20, 160)
surf = ax.plot_trisurf(ari[:,0], ari[:,1], ari[:,2], cmap=plt.cm.coolwarm, linewidth=0, antialiased=False)
plt.tight_layout()
plt.show()

**For `make_blobs()` with the default values:** Given the range of the parameters, the value for `eps` has clearly a greater effect on the result compared to `min_samples`. This is due to the blobs reasonably well separated and of similar density. As soon as an increasing `eps` value crosses a threshold, 2 clusters merge into one -- reflected in the step-by-step reduction in the rand index.

Now the plot for the silhouette scores.

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.set_xlabel(r'min_samples', fontsize=16)
ax.set_ylabel(r'eps', fontsize=16)
ax.set_zlabel('Silhouette', fontsize=16)
ax.view_init(20, 160)
surf = ax.plot_trisurf(silhouette[:,0], silhouette[:,1], silhouette[:,2], cmap=plt.cm.coolwarm, linewidth=0, antialiased=False)
plt.tight_layout()
plt.show()

**For `make_blobs()` with the default values:** The silhouette score is more stable than the rand index for a range of parameter values, since the calculation of the silhouette score only considers cluster points but NOT noise points. Whether an outer point of a blob is part of the cluster or not, does not make a big difference.

By finding the parameter setting with the highest rand index, we can perform DBSCAN with these setting and plot the results.

In [ ]:
best_run = np.argmax(ari[:,2])
best_min_samples = int(ari[:,0][best_run])
best_eps = ari[:,1][best_run]

dbscan = DBSCAN(eps=best_eps, min_samples=best_min_samples).fit(X_demo) 
plot_clusters(dbscan, X_demo)

And we can do the same thing using the silhoutte scores.

In [ ]:
best_run = np.argmax(silhouette[:,2])
best_min_samples = int(silhouette[:,0][best_run])
best_eps = silhouette[:,1][best_run]

dbscan = DBSCAN(eps=best_eps, min_samples=best_min_samples).fit(X_demo) 
plot_clusters(dbscan, X_demo)


Note that for the non-blob datasets, the parameter setting resulting in the highest silhouette score does yield the two intuitively preferred clusters. The main reason is that the silhouette score, similar to SSE, favors blob-like clusters.

---

## DBSCAN over a Real-World Dataset

The [Pima Indians Diabetes Dataset](https://www.kaggle.com/uciml/pima-indians-diabetes-database) is a well-known dataset frequently used in machine learning and data mining research. It contains information about a group of Pima Indian women from Arizona, USA, and their potential risk of developing diabetes. The dataset is commonly used for classification tasks, aiming to predict whether a person has diabetes or not based on various features.

Here are the details of the Pima diabetes dataset:

* Number of instances: 768
* Number of attributes: 8
* Target variable: Outcome (0 for non-diabetic, 1 for diabetic)

The attributes or features in the dataset are:

* Pregnancies: Number of times pregnant.
* Glucose: Plasma glucose concentration after a 2-hour oral glucose tolerance test.
* Blood Pressure: Diastolic blood pressure (mm Hg).
* Skin Thickness: Triceps skinfold thickness (mm).
* Insulin: 2-Hour serum insulin (mu U/ml).
* BMI: Body mass index (weight in kg / (height in m)^2).
* Diabetes Pedigree Function: Diabetes pedigree function, which provides an estimation of the genetic influence.
* Age: Age in years.

This is a convenient dataset since all attributes for the clustering are numerical, and we can use the default Euclidean distance as a similarity measure. First, as usual, we read the dataset from the file into a `pandas` DataFrame.

In [ ]:
df_diabetes = pd.read_csv('data/datasets/diabetes/diabetes.csv')

df_diabetes.head()

**Important:** Remember, in practice, you should always do at least a basic EDA to check for missing values and (obvious) outliers! We skip this step to keep the notebook simple.

First, we need to create the input data `X` for the clustering and -- for this example -- the label data `y`.

In [ ]:
X_diabetes = df_diabetes.drop(columns=['Outcome']).to_numpy()
y_diabetes = df_diabetes['Outcome'].to_numpy()

Optionally, we can normalize the data, e.g., using standardization. In fact, it is highly recommended that we do so as the domains of different attributes differ by several orders of magnitude. You can check again with the EDA and Data Preprocessing notebook regarding the importance of normalizing input features.

In [ ]:
scaler = StandardScaler()
X_diabetes = scaler.fit_transform(X_diabetes)

### Estimate Parameter Settings

Selecting meaningful parameters for `eps` and `min_samples` is generally not obvious. One basic approach to get some basic sense for meaningful `eps` values is to look at the distribution of pairwise distances between all data points.

In [ ]:
# scikit-learn makes it very convenient to calculate pairwise distances
dist = euclidean_distances(X_diabetes, X_diabetes)

# Matrix dist is symmentrical so we can ignore half of it
# (including the diagonal where all values a 0)
dist = dist[np.triu_indices_from(dist, k=1)]

Let's use a histogram to plot the distribution of pairwise lengths.

In [ ]:
plt.figure()
plt.tick_params(labelsize=14)
plt.hist(dist, bins=200)
plt.xlabel('pairwise distance', fontsize=16)
plt.ylabel('count', fontsize=16)
plt.tight_layout()
plt.show()

This plot gives us a crude idea how to set `eps`. For example, values (far) above the average are likely to yield a single big cluster; try `eps=4`. On the other hand, values close to 0 are arguably too small to result in too many noise points and very small clusters. Note that these values will also depend on whether the attributes have been standardized or not!!!

Let's run DBSCAN with some first crude choices for `eps` and `min_samples`.

In [ ]:
dbscan_diabetes = DBSCAN(eps=1, min_samples=5).fit(X_diabetes) 

print(np.unique(dbscan_diabetes.labels_))

### Visualization

As our dataset now has more than 3 attributes, we cannot directly plot the clusters. However, we can use dimensionality reduction techniques to map the data points into a lower-dimensional space. In this example, we use PCA to map our data into 2d -- a detailed discussion about PCA is beyond the scope of this notebook.

In [ ]:
X_diabetes_pca = PCA(n_components=2).fit_transform(X_diabetes)
X_diabetes_tsne = TSNE(n_components=2, init='random').fit_transform(X_diabetes)

In [ ]:
plot_clusters(dbscan_diabetes, X_diabetes_pca)

In [ ]:
plot_clusters(dbscan_diabetes, X_diabetes_tsne)

As you will have noticed, some of the clusters do not look as you would expect and you have seen in the plots above. For example some noise points might be very close to cluster points. The difference is that we performed the clustering by running DBSCAN in the 3d space and then used PCA to map the data into the 2d space. The issue is that dimensionality reduction always results in some loss of information. Here, we lose the separation of clusters in the plot, but only in the plot!

---

## Location Data

DBSCAN is particularly interesting for datasets where we already know or expect the data not to be blobs but of different shapes. A good example is location data, where bus stops or bars might cluster along a popular road. Of course, bars or restaurants might also occur in the shape of blobs in popular areas.

The following CSV file contains a list of places/venues with their type and geographic location in terms of latitude-longitude pairs.

In [ ]:
# This CSV file has not header line at the top, so the columns will be unnamed
# but if you look at the data you should identify the meaning of each column
df = pd.read_csv('data/datasets/singapore/sg-places.csv', header=None)

Let's have a look at all possible values for Column 2, which reflects the type of a place, just to see what's available to us. You can of course consider all places, but things get quickly too cluttered.

In [ ]:
print(set(df[2]))

Let's pick all the places of one type. The descriptions in the following well refer to restaurants (in more detail: McDonald's restaurants; see below). But feel free to change the type to anything of your interest.

In [ ]:
# Try different place types
place_type = 'restaurant'
#place_type = 'bus_station'
#place_type = 'subway_station' # MRT+LRT stations
#place_type = 'store'

df_places = df[df[2]==place_type]

print('Number of places: {}'.format(df_places.shape[0]))

df_places.head()

3k+ restaurants is quite a lot, and maybe even too diverse for an analysis. So let's further filter the dataset to only include McDonald's restaurants. We use regular expressions for that. No worries, if you're not familiar with regular expressions. In a nutshell, they allow for filtering using flexible substring matching.

In [ ]:
p = re.compile(r'mcdonald', flags=re.IGNORECASE)
df_places = df_places[[bool(p.search(x)) for x in df_places[0]]]

df_places.head()

### Visualization

In [ ]:
num_restaurants, num_attributes = df_places.shape

print('Number of places: {}'.format(num_restaurants))

We need to convert the column containing latitude and longitude to a matrix, i.e., a 2d numpy array for further processing. Note that the resulting order is \[longitude, latitude\], since longitude represents the x variable and latitude the y variable. This doesn't matter for the clustering but it ensures that the plots look alright and are not rotated by 90 degrees.

In [ ]:
X_places = df_places[[4, 3]].to_numpy()

print(X_places[0])

The code cell below provides a neat way that the proportions of the plotted points look nicer. Otherwise, the induced shape of Singapore will be squashed. Since Singapore is so close to the equator, this correction is not really needed, though.

In [ ]:
aspect = 1/np.cos(np.radians(1.35))
print(aspect)

Let's plot all the places (e.g., the 141 McDonald's restaurant). You should be able to recognize the outline of Singapore. Of course, if you pick place types that are much less common and/or can only be found in certain areas, you won't be able to "see" the outline of Singapore.

In [ ]:
plt.figure()
plt.axes().set_aspect(aspect)
plt.scatter(X_places[:,0], X_places[:,1], s=25)
plt.tick_params(top=False, bottom=False, left=False, right=False, labelleft=False, labelbottom=False)  
plt.tight_layout()
plt.show()

### Performing DBSCAN

By default, the DBSCAN implementation of scikit-learn, would treat the latitude-longitude pairs as coordinates in the Euclidean space. DBSCAN would still work fine, but we would have a hard time to pick intuitive values for `eps`. The Euclidean distance between two geolocations has no meaningful unit such as meter or kilometer.

Luckily, DBSCAN allows us to provide a custom similarity/distance metric. The method below calculates the distance between two geolocations in meters; this function can be found on the Web.

In [ ]:
def geo_distance_in_meters(point1, point2):
    lon1, lat1 = point1[0], point1[1]
    lon2, lat2 = point2[0], point2[1]
    dlon = np.radians(lon2) - np.radians(lon1)
    dlat = np.radians(lat2) - np.radians(lat1)

    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

    distance_haversine_formula = 6371000 * c
    return distance_haversine_formula

Now we are ready to run DBSCAN; notice how the method for calculating distances is given as a parameter. With this, we can now express `eps` in meters, making it much more intuitive to pick suitable values -- note that this is a case where we do not want to normalize/standardize the data.

In the example below, we set `eps=500` and `min_samples=5`, which roughly translates to: find clusters with 5 or McDonald's restaurants in a vicinity of 1km (circle with a radius of 500m).

In [ ]:
dbscan_places = DBSCAN(metric=geo_distance_in_meters, eps=500, min_samples=5).fit(X_places) 

# The set of unique quickly labels shows how many clusters there are; -1 is noise
print(np.unique(dbscan_places.labels_))

Let's plot the clusters to see where the McDonald's centers of Singapore are :).

In [ ]:
plot_clusters(dbscan_places, X_places, show_ticks=False, point_size=50, aspect=aspect)

There are different Python packages to allow for a fancier plotting of geolocations on top of actual maps. But those are "only" visualization details which are beyond this notebook. But note that good visualizations are an important part of data mining to interpret and share results.

---

## Summary

DBSCAN (Density-Based Spatial Clustering of Applications with Noise) is a density-based clustering algorithm that identifies clusters in a dataset based on the density distribution of data points. It offers several advantages that make it a popular choice for clustering tasks.

One of the main advantages of DBSCAN is its ability to discover clusters of arbitrary shape. Unlike algorithms like k-means, which assume clusters to be spherical, DBSCAN can identify clusters of varying shapes and sizes. This makes it more suitable for real-world datasets where clusters may not conform to strict geometric assumptions.

Another advantage is its robustness to outliers and noise. DBSCAN can effectively handle noise points and classify them as outliers, as they do not belong to any cluster. This is particularly useful in scenarios where the dataset may contain incomplete or noisy data.

Furthermore, DBSCAN does not require the number of clusters to be predefined. It automatically determines the number of clusters based on the density of the data points. This flexibility makes it suitable for datasets where the number of clusters is unknown or may vary.

Despite its advantages, DBSCAN does have some limitations. It requires the appropriate choice of parameters, such as the radius (eps) and the minimum number of points (min_samples) for a point to be considered a core point. Choosing these parameters can be challenging and may impact the clustering results. Additionally, DBSCAN may struggle with datasets of varying densities, as it assumes a uniform density across clusters. If the density varies significantly, it may result in uneven cluster sizes or failure to identify smaller clusters.

In summary, DBSCAN is a powerful clustering algorithm that can discover clusters of arbitrary shape, handle outliers, and automatically determine the number of clusters. It is particularly useful for real-world datasets and offers more flexibility compared to traditional clustering algorithms. However, it requires careful parameter selection and may struggle with datasets of varying densities.
